In [1]:
import pandas as pd
import numpy as np 
from random import shuffle
import matplotlib.pyplot as plt
import csv
df = pd.read_csv('train/train_1.csv')
df = df.fillna(0)

In [3]:
print(df.head())

                                                Page  2015-07-01  2015-07-02  \
0            2NE1_zh.wikipedia.org_all-access_spider        18.0        11.0   
1             2PM_zh.wikipedia.org_all-access_spider        11.0        14.0   
2              3C_zh.wikipedia.org_all-access_spider         1.0         0.0   
3         4minute_zh.wikipedia.org_all-access_spider        35.0        13.0   
4  52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...         0.0         0.0   

   2015-07-03  2015-07-04  2015-07-05  2015-07-06  2015-07-07  2015-07-08  \
0         5.0        13.0        14.0         9.0         9.0        22.0   
1        15.0        18.0        11.0        13.0        22.0        11.0   
2         1.0         1.0         0.0         4.0         0.0         3.0   
3        10.0        94.0         4.0        26.0        14.0         9.0   
4         0.0         0.0         0.0         0.0         0.0         0.0   

   2015-07-09     ...      2016-12-22  2016-12-23  2016-

In [12]:
data_len = len(df.columns)-1
content_len = 5000
requests = []
df_values = df.values[:,1:]
with open("requests5000.csv", "w") as f:
        writer = csv.writer(f)
        for day in range(data_len):
            print(day)
            day_reqs = []
            for content_type in range(content_len):
                day_reqs += [content_type] * int(df_values[content_type,day])
            shuffle(day_reqs)
            writer.writerow(day_reqs)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [67]:
import sys
sys.path.insert(0, './src')
from router import Router
from consumer import Consumer
from producer import Producer
from graph import Graph
from packet import Packet
from test import *
import random
import math
import numpy as np
from matplotlib import pyplot as plt
from eventlist import *
import pandas as pd 


"""Creates visualization for simulation"""
def visualize(adj_mtx, consumers, producers):
    import matplotlib.pyplot as plt
    import networkx as nx
    gr = nx.Graph()
    for i in range(len(adj_mtx)):
        gr.add_node('r' + str(i))
    for i in range(len(adj_mtx)):
        for j in range(len(adj_mtx[i])):
            if adj_mtx[i][j] == 1:
                gr.add_edge('r' + str(i), 'r' + str(j))
    for c in consumers:
        gr.add_node(c.name)
        gr.add_edge(c.name, c.gateway.name)
    for p in producers:
        gr.add_node(p.name)
        gr.add_edge(p.name, p.gateway.name)
    nx.draw(gr, node_size = 500, with_labels = True)
    plt.show()


""" Creates CCN Simulation for a given network scenaario

    Attributes:
        NUM_REQUESTS_PER_CONSUMER: num pkt requests each consumer will make
        REQUEST_RATE:
        ZIPF_S: Parameter for Zipf Distribution
        NUM_CONTENT_TYPES: num of unique pkt names in network
        CACHE_SIZE: size of router caches
        TIME_STEP: length of timestep interval
        consumers: list of all consumer nodes
        producers: list of all producer nodes
        net_core: Container for interfacing with router nodes
        zipf_weights: list of zipf distribution based probabilities for content_types
"""
class Simulator:

    def __init__(self, 
                num_consumers, 
                num_content_types, 
                end_day, 
                request_rate,
                zipf_s,
                m_q,
                num_cycles,
                zipf_update_interval, 
                cache_update_interval,
                grid_rows, grid_cols, 
                cache_ratio, 
                policy, 
                requests,
                max_reqs_per_day,
                rand_seed = 123):

#         self.ZIPF_S = zipf_s                                            # zipf skew
        self.M_Q = m_q                                                  # mandelbrot q
        self.NUM_CYCLES = num_cycles                                    # number of zipf cycles
        self.REQUEST_RATE = request_rate                                # requests/s for each consumer
        self.NUM_CONTENT_TYPES = num_content_types                      # number of unique content types
        self.CACHE_SIZE = int(cache_ratio * self.NUM_CONTENT_TYPES)     
        self.CACHE_UPDATE_INTERVAL = cache_update_interval              # interval to call update_state() on caches
        self.ZIPF_UPDATE_INTERVAL = zipf_update_interval                # interval to switch zipf distributions
        self.RAND_SEED = rand_seed
        self.policy = policy
        self.requests = requests
        self.max_reqs_per_day = max_reqs_per_day
        self.curr_request = 0
        print(len(self.requests))
        print(len(self.requests[0]))

        random.seed(self.RAND_SEED)
        np.random.seed(self.RAND_SEED)

        self.rng = np.random.RandomState(self.RAND_SEED)                # random number generator responsible for all non third-party randomizations

#         self.history = []                                               # history of requests throughout simulation
#         for i in range(self.NUM_CYCLES):
#             self.history.append([])

        self.q = EventList()                                            # global queue of events
#         self.zipf_cycle = 0                                             # counter to switch zipf distributions
        self.prev_cache_update = 0                                      
#         self.prev_zipf_update = 0
        self.timestep =0
        self.curr_time = 0
        self.curr_day = 0                                              # continuously increasing time
        self.end_day = end_day
        self.consumers = []
        self.producers = []
       
        num_routers = grid_rows * grid_cols                             # number of total routers in topology

        # populate content
        self.content_types = ["content" + str(i) \
            for i in range(self.NUM_CONTENT_TYPES)] 
        
        # initialize routers
        self.net_core = Graph(
                            self.CACHE_SIZE, 
                            self.NUM_CONTENT_TYPES, 
                            grid_rows, 
                            grid_cols, 
                            self.policy, 
                            self.q, 
                            self.rng
                        )

        # assign consumers and producers with gateway routers
        for i in range(num_consumers):
            r = self.net_core.get_random_router()
            self.consumers.append(
                Consumer("c" + str(i), r, self.q)
            )

        for i in range(num_content_types):
            r = self.net_core.get_random_router()
            self.producers.append(
                Producer("p" + str(i), r, "content" + str(i), self.q)
            )

        # set FIBs in routers
        self.net_core.set_routes_to_producers(self.producers)
    
    def get_next_actor(self):
        """Returns next actor (node) to execute event for (event with min value for time)"""
        min_time = None
        actor = None
        actor_name, min_time = self.q.peek()                    # get next event from queue
        if actor_name != None:                                  # if q is not empty
            actor_ix = int(actor_name[1:])      
            actor_type = actor_name[0]
            if actor_type == 'c':                               # if consumer
                actor = self.consumers[actor_ix]
                self.set_next_content_request(actor)            # schedule next consumer request,
                                                                # ensures queue is never empty and simulation
                                                                # will run until end time

            elif actor_type == 'r':                             # if router
                actor = self.net_core.routers[actor_ix]
            else:                                               # if producer
                actor = self.producers[actor_ix]
            assert(self.curr_time <= min_time)                  # events cannot occur in the past
        self.curr_time = min_time
        return actor
    

    def set_next_content_request(self, consumer):
        """Append (time, req, packet) pair into consumer queue"""
        if self.max_reqs_per_day:
            if self.curr_request == self.max_reqs_per_day: # max reqs per day. increment day
                self.curr_request = 0
                self.curr_day += 1
                self.timestep +=1
        elif self.curr_request == len(self.requests[self.timestep]): #entire reqs for the day done
            self.curr_request = 0
            self.curr_day += 1
            self.timestep += 1
            
        if self.timestep >= len(self.requests): # window change. increment window. get new reqs
            self.timestep = 0
            self.curr_request = 0
            self.curr_day +=1
            self.requests = get_new_requests(self.curr_day,self.CACHE_UPDATE_INTERVAL)
        content_name = "content"+str(self.requests[self.timestep][self.curr_request])
        print(content_name)
        self.curr_request +=1
        self.q.add(Event(consumer.name,consumer.time_of_next_request,'REQ',Packet(content_name),None))
        consumer.time_of_next_request += self.rng.exponential(1/self.REQUEST_RATE)
#         self.history[self.zipf_cycle % self.NUM_CYCLES].append(content_name) # update sim history

    def run(self):
        """Executes events for nodes"""
        for consumer in self.consumers:                 # init simulation with a content request from all consumers
            self.set_next_content_request(consumer)
        actor = self.get_next_actor()
        while self.curr_day < self.end_day:
            # print(self.curr_time)
            if self.curr_day - self.prev_cache_update > self.CACHE_UPDATE_INTERVAL:
                self.prev_cache_update = self.curr_time
                for ix, router in enumerate(self.net_core.routers):
                    router.contentstore.update_state()
                self.requests = get_new_requests(self.curr_day,self.CACHE_UPDATE_INTERVAL)
                self.timestep = 0
#             if self.curr_time - self.prev_zipf_update > self.ZIPF_UPDATE_INTERVAL:
#                 self.prev_zipf_update = self.curr_time
#                 self.zipf_cycle += 1
            actor.execute()
            actor = self.get_next_actor()
            
        # visualize(self.net_core.adj_mtx, self.consumers, self.producers)

# if __name__ == "__main__":
def simulate(num_content_types,total_days,window):
    RAND_SEED = 123
    """ Simulation Sample Scenario """
    for policy in ['lfu']:
        sim = Simulator(
            num_consumers=1, 
            num_content_types=num_content_types, 
            end_day=total_days,
            request_rate=1,
            zipf_s=0.7,
            m_q=0.7,
            num_cycles=3,
            zipf_update_interval = 0,
            cache_update_interval=window, #day(s)
            grid_rows=1, 
            grid_cols=1, 
            cache_ratio=0.3, 
            policy=policy, 
            rand_seed=RAND_SEED,
            requests = get_new_requests(0,window),
            max_reqs_per_day = 1000 # set to False if not required
        )
    sim.run()

#         """Optimal Hit Rate (Theoretical Maximum)"""
        
#         num_requests = sum([len(hist) for hist in sim.history])

#         num_hits = 0
#         for i, hist in enumerate(sim.history):
#             opt_contents = sorted(enumerate(sim.zipf_set[i]), key = lambda x: x[1], reverse=True)
#             opt_cache = ['content' + str(ix) for ix, pop in opt_contents[:int(sim.CACHE_SIZE)]]
#             for o in opt_cache:
#                 for h in hist:
#                     if h == o:
#                         num_hits += 1
#         print('Optimal Hit Rate: ', num_hits/num_requests)
        
#         """"""

#         """ Policy Hit Rates """

#         for router in sim.net_core.routers:
#             total = router.contentstore.hits + router.contentstore.misses
#             if total:
#                 print(router.name + ' ' + sim.policy + ' Hit Rate: ' + str(router.contentstore.hits/total))     

#         """""" 
simulate(5000,549,2)

def get_new_requests(from_day,window):
    with open('requests5000.csv') as fd:
        reader=csv.reader(fd)
        days = list(range(from_day,from_day+window))
        print("days",days)
        interestingrows=[row for idx, row in enumerate(reader) if idx in days]
        print(len(interestingrows))
        return interestingrows

days [0, 1]
2
2
278503
content4929
content4344
content4856
content4578
content4664
content1210
content4877
content4712
content4472
content4586
content911
content4698
content4763
content4466
content4600
content1277
content4471
content4497
content4429
content4508
content4441
content1440
content4464
content4989
content4745
content1791
content3037
content4995
content4939
content450
content2860
content4348
content4929
content4679
content4666
content4992
content3051
content865
content4470
content4965
content4459
content4500
content4778
content4439
content4456
content4529
content2136
content4473
content4484
content4534
content4639
content4755
content4350
content4889
content138
content4855
content4330
content4912
content4583
content4645
content4662
content4593
content4645
content4873
content4381
content4918
content1508
content1332
content4651
content4413
content4607
content4447
content4403
content4555
content4397
content4736
content4366
content4793
content4938
content4742
content796
content445

content4777
content4401
content4391
content3160
content4709
content4991
content3020
content4690
content4667
content4861
content4692
content4882
content4955
content4820
content4475
content782
content4334
content4403
content4378
content1213
content4507
content4621
content3156
content4990
content4800
content2679
content782
content4880
content4442
content4915
content4995
content4442
content4646
content982
content3357
content4344
content4967
content4974
content2918
content4406
content4600
content4363
content4872
content865
content4671
content4787
content4365
content4569
content4820
content4475
content745
content4825
content3380
content4386
content418
content4353
content4514
content4783
content4545
content4442
content4326
content4777
content4891
content4505
content4579
content4397
content4974
content4920
content4344
content2454
content4350
content4929
content4512
content4334
content4645
content3218
content4929
content4942
content4774
content4762
content4589
content4485
content4311
content443

2
content4680
days [3, 4]


KeyboardInterrupt: 

In [1]:
import sys; sys.path

['/home/ghoshubu/Documents/fyp/venv/lib/python36.zip',
 '/home/ghoshubu/Documents/fyp/venv/lib/python3.6',
 '/home/ghoshubu/Documents/fyp/venv/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '',
 '/home/ghoshubu/Documents/fyp/venv/lib/python3.6/site-packages',
 '/home/ghoshubu/Documents/fyp/venv/lib/python3.6/site-packages/IPython/extensions',
 '/home/ghoshubu/.ipython']